[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/arena-ai/structured-logprobs/blob/main/docs/notebooks/notebook.ipynb)

This notebook provides a practical guide on how to use the structured-logprobs library alongside the OpenAI API to generate structured responses with log probability features.

## Install the library

`structured-logprobs` is distributed on PyPI and simply installed with pip install.

In [ ]:
!pip install structured-logprobs

Let's import the required libraries.

In [2]:
import getpass

from openai import OpenAI
from openai.types import ResponseFormatJSONSchema

from structured_logprobs.main import add_logprobs, add_logprobs_inline

## Setting Up the OpenAI API Client

An OpenAI API key is mandatory to authenticates the access to OpenAI's API. It serves as a token necessary to initialize the OpenAI client in Python, enabling you to send requests to the API and receive responses.

In this notebook, you'll be prompted to enter your OPENAI_API_KEY securely using Python's getpass module. This ensures that your key is not hardcoded, reducing the risk of accidental exposure.

In [ ]:
api_key = getpass.getpass(prompt="Enter you OPENAI_API_KEY: ")

Let's initialize the OpenAI client.

In [4]:
client = OpenAI(api_key=api_key)

## Create a chat completion request

The first step is to define the JSON schema, which will be used to structure the chat request to OpenAI. This schema helps OpenAI understand exactly how the response should be formatted.

Below is the example of the JSON file that defines the schema used for validating the responses.

In [ ]:
schema_content = {
    "type": "json_schema",
    "json_schema": {
        "name": "answears",
        "description": "Response to questions in JSON format",
        "schema": {
            "type": "object",
            "properties": {
                "capital_of_France": {"type": "string"},
                "the_two_nicest_colors": {
                    "type": "array",
                    "items": {"type": "string", "enum": ["red", "blue", "green", "yellow", "purple"]},
                },
                "die_shows": {"type": "integer"},
            },
            "required": ["capital_of_France", "the_two_nicest_colors", "die_shows"],
            "additionalProperties": False,
        },
        "strict": True,
    },
}

The schema must be validated before being used as a parameter in the request to OpenAI to ensure it is correctly structured as a valid JSON.

In [6]:
response_schema = ResponseFormatJSONSchema.model_validate(schema_content)

Additionally, to create the chat completion, you must set up the model, input messages, and other parameters such as logprobs and response_format.

In [7]:
completion = client.chat.completions.create(
    model="gpt-4o-2024-08-06",
    messages=[
        {
            "role": "system",
            "content": (
                "I have three questions. The first question is: What is the capital of France? "
                "The second question is: Which are the two nicest colors? "
                "The third question is: Can you roll a die and tell me which number comes up?"
            ),
        }
    ],
    logprobs=True,
    response_format=response_schema.model_dump(by_alias=True),
)

In [24]:
print(completion)

ChatCompletion(id='chatcmpl-ApBLeOHlpf3mbFIgVsaMy1V4Y0aB0', choices=[Choice(finish_reason='stop', index=0, logprobs=ChoiceLogprobs(content=[ChatCompletionTokenLogprob(token='{"', bytes=[123, 34], logprob=-6.6306106e-06, top_logprobs=[]), ChatCompletionTokenLogprob(token='capital', bytes=[99, 97, 112, 105, 116, 97, 108], logprob=-3.0545007e-06, top_logprobs=[]), ChatCompletionTokenLogprob(token='_of', bytes=[95, 111, 102], logprob=0.0, top_logprobs=[]), ChatCompletionTokenLogprob(token='_F', bytes=[95, 70], logprob=-1.9361265e-07, top_logprobs=[]), ChatCompletionTokenLogprob(token='rance', bytes=[114, 97, 110, 99, 101], logprob=-3.1281633e-07, top_logprobs=[]), ChatCompletionTokenLogprob(token='":"', bytes=[34, 58, 34], logprob=0.0, top_logprobs=[]), ChatCompletionTokenLogprob(token='Paris', bytes=[80, 97, 114, 105, 115], logprob=-9.0883464e-07, top_logprobs=[]), ChatCompletionTokenLogprob(token='","', bytes=[34, 44, 34], logprob=-1.9361265e-07, top_logprobs=[]), ChatCompletionTokenLogp

## Enhance the chat completion result with log probabilities

In [ ]:
chat_completion = add_logprobs(completion)

Let's print the original chat completion response.

In [12]:
print(chat_completion.value.choices[0].message.content)

{"capital_of_France":"Paris","the_two_nicest_colors":["blue","green"],"die_shows":4}


Let's print the log_probs field, structured like the message of the response, but where values are replaced with their respective log-probabilities.

In [13]:
print(chat_completion.log_probs)

[{'capital_of_France': -5.5122365e-07, 'the_two_nicest_colors': [-0.00283229711265, -0.01072703461265], 'die_shows': -0.43976903}]


## Enhance the chat completion result with in-line log probabilities

In [14]:
chat_completion_inline = add_logprobs_inline(completion)

This method modifies the chat completion response object. The content of the message is replaced with a dictionary that includes also inline log probabilities for atomic values.

In [17]:
print(chat_completion_inline.choices[0].message.content)

{"capital_of_France": "Paris", "capital_of_France_logprob": -5.5122365e-07, "the_two_nicest_colors": ["blue", "green"], "die_shows": 4.0, "die_shows_logprob": -0.43976903}
